In [32]:
from langgraph.graph import StateGraph , START , END
from typing import TypedDict , Literal
from pydantic import BaseModel , Field
from langchain_ollama import ChatOllama

In [33]:
llm = ChatOllama(model = 'llama3')

In [34]:
#state
class ReviewState(TypedDict):
    review : str
    sentiment : Literal['positive' , 'negative']
    diagnosis : dict
    response : str

In [35]:
class SentimentSchema(BaseModel):
    sentiment : Literal['positive' , 'negative'] = Field(description = 'Sentiment of the review')

In [36]:
class DiagnosisSchema(BaseModel):
    issue_type: Literal["UX", "Performance", "Bug", "Support", "Other"] = Field(description='The category of issue mentioned in the review')
    tone: Literal["angry", "frustrated", "disappointed", "calm"] = Field(description='The emotional tone expressed by the user')
    urgency: Literal["low", "medium", "high"] = Field(description='How urgent or critical the issue appears to be')

In [37]:
structured_llm2 = llm.with_structured_output(DiagnosisSchema)

In [38]:
structured_llm = llm.with_structured_output(SentimentSchema)

In [39]:
def sentiment(state : ReviewState) -> ReviewState :
    prompt = f"For the following review,find out the sentiment /n {state['review']}"

    output = structured_llm.invoke(prompt).sentiment

    return {'sentiment' : output}

In [40]:
def check_condition(state : ReviewState) -> Literal['positive_response' , 'diagnosis']:
    sentiment = state['sentiment']

    if sentiment == 'positive':
        return 'positive_response'
    else :
        return 'diagnosis'

In [41]:
def positive_response(state : ReviewState) -> ReviewState:
    prompt = f"""Write a warm thank-you message in response to this review:
    \n\n\"{state['review']}\"\n
    Also, kindly ask the user to leave feedback on our website."""

    output = llm.invoke(prompt).content

    return {'response' : output}

In [42]:
def diagnosis(state : ReviewState) -> ReviewState:
    prompt = f"""Diagnose this negative review:\n\n{state['review']}\n"
    "Return issue_type, tone, and urgency.
    """

    response = structured_llm2.invoke(prompt)

    return {'diagnosis' : response.model_dump()}

In [43]:
def negative_response(state : ReviewState) -> ReviewState:
    diagnosis = state['diagnosis']

    prompt = f"""You are a support assistant.
    The user had a '{diagnosis['issue_type']}' issue, sounded '{diagnosis['tone']}', and marked urgency as '{diagnosis['urgency']}'.
    Write an empathetic, helpful resolution message.
    """

    response = llm.invoke(prompt).content

    return {'response' : response}

In [44]:
#graph
graph = StateGraph(ReviewState)

graph.add_node('sentiment' , sentiment)
graph.add_node('positive_response' , positive_response)
graph.add_node('negative_response' , negative_response)
graph.add_node('diagnosis' , diagnosis)

graph.add_edge(START , 'sentiment')
graph.add_conditional_edges('sentiment' , check_condition)
graph.add_edge('positive_response' , END)
graph.add_edge('diagnosis' , 'negative_response')
graph.add_edge('negative_response' , END)

workflow = graph.compile()


In [45]:
#execute 

intial_state={
    'review': "I’ve been trying to log in for over an hour now, and the app keeps freezing on the authentication screen. I even tried reinstalling it, but no luck. This kind of bug is unacceptable, especially when it affects basic functionality."
}
workflow.invoke(intial_state)

{'review': 'I’ve been trying to log in for over an hour now, and the app keeps freezing on the authentication screen. I even tried reinstalling it, but no luck. This kind of bug is unacceptable, especially when it affects basic functionality.',
 'sentiment': 'negative',
 'diagnosis': {'issue_type': 'Support',
  'tone': 'frustrated',
  'urgency': 'high'},
 'response': "Dear [User],\n\nI'm so sorry to hear that you're experiencing a high-urgency Support issue. I can imagine how frustrating it must be for you, and I want to assure you that we're here to help.\n\nFirstly, please know that your frustration is valid, and I'm here to listen and assist you in resolving this issue as quickly as possible. Your prompt attention is crucial, and I'll do my best to expedite the process.\n\nTo better understand your situation, could you please provide me with more details about what's not working the way it should? What specific steps have you taken so far, and what error messages or issues are you e